In [23]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
import keras
from keras.layers import Input, Dense, Conv1D, MaxPooling1D, GlobalAveragePooling1D, Reshape, Flatten, BatchNormalization, LSTM
from keras.optimizers import SGD

from sklearn.preprocessing import Imputer, OneHotEncoder, StandardScaler
from sklearn import preprocessing
import numpy as np
import pandas as pd
from scipy import stats

import time

import os
import os.path
from pathlib import Path

from pandas import Series
from sklearn.preprocessing import MinMaxScaler

np.random.seed(1337)

_script_path = Path().absolute() #location of our script
_dataset_folder_name = 'filtered_data'
_dataset_folder_path = os.path.join(str(_script_path), _dataset_folder_name)

_file_names = []
_folder_locations = []  
_dataset_list = []

for dirpath, dirnames, filenames in os.walk(_dataset_folder_path):
    for filename in [f for f in filenames if f.endswith(".csv")]:
        location = os.path.join(dirpath, filename)
        _folder_locations.append(location)
        _file_names.append(filename)

for i,location in enumerate(_folder_locations):
    temp_df = pd.read_csv(location,engine='python')
    values = temp_df.iloc[:,0].values
    values = values.reshape((len(values), 1))
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(values)
    normalized = scaler.transform(values)
    temp_df = temp_df.drop(['ecg'],axis = 1)
    temp_df['ecg'] = normalized
    temp_df = temp_df.truncate(after = 17999)
    temp_df['participant_no'] = i
    if "anger" in location:
        temp_df['emotion'] = 0
    elif "calmness" in location:
        temp_df['emotion'] = 1
    elif "disgust" in location:
        temp_df['emotion'] = 2
    elif "fear" in location:
        temp_df['emotion'] = 3
    elif "happiness" in location:
        temp_df['emotion'] = 4
    elif "sadness" in location:
        temp_df['emotion'] = 5
    unc_columns = ['timest']
    temp_df = temp_df.drop(unc_columns,axis=1)
    _dataset_list.append(temp_df)

_dataset = pd.concat(_dataset_list,axis=0)
_dataset.index = range(0,len(_dataset))
_dataset = _dataset.sample(frac=1).reset_index(drop=True)
train_x = _dataset.iloc[:,0:5]
#train_x = train_x.drop(['participant_no'],axis=1)
train_y = _dataset.iloc[:,6:]
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)

print('Row count= ', len(_dataset))

train_x = train_x.reshape(312,18000,5)
train_y = train_y.values.reshape(312,18000)
trunc_train_y = train_y[:,:1]


train_y_enc = pd.DataFrame(trunc_train_y)
train_y_enc = pd.get_dummies(train_y_enc[0])

/home/mert/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/mert/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


Row count=  5616000


In [24]:
train_x_list = []
train_y_list = []
previous_value = 0
while previous_value<= 17599:
    train_x_list.append(train_x[:,previous_value:previous_value+400,:])
    #train_y_list.append(train_y_enc.values[previous_value:previous_value+400,:])
    previous_value+=100

In [25]:
train_x_list[3].shape

(312, 400, 5)

In [ ]:
model = Sequential()
model.add(Conv1D(5,100,activation='relu',input_shape=(400,5)))
model.add(LSTM(32,return_sequences=True,dropout=0.2,recurrent_dropout=0.2))
model.add(LSTM(64,return_sequences=True,dropout=0.2,recurrent_dropout=0.2))
model.add(LSTM(32,return_sequences=False))
model.add(Dense(6,activation='softmax'))

print(model.summary)

rmsprop = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
#sgd = keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
sgd = keras.optimizers.SGD(lr=0.000001, clipvalue=0.5)
adagrad = keras.optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)
adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['acc'])

for i in range(50):
    print('Training ', i, 'th window.')
    history = model.fit(train_x_list[i],train_y_enc,epochs = 5, batch_size = 32, validation_split=0.33)
    #model.reset_states()



<bound method Network.summary of <keras.engine.sequential.Sequential object at 0x7f3e4d61c5f8>>
Training  0 th window.
Train on 209 samples, validate on 103 samples
Epoch 1/5
209/209 [==============================] - 8s 39ms/step - loss: 1.8389 - acc: 0.1579 - val_loss: 1.7866 - val_acc: 0.1262
Epoch 2/5
209/209 [==============================] - 4s 19ms/step - loss: 1.7802 - acc: 0.2392 - val_loss: 1.8269 - val_acc: 0.2039
Epoch 3/5
209/209 [==============================] - 4s 19ms/step - loss: 1.6780 - acc: 0.2919 - val_loss: 1.9832 - val_acc: 0.2233
Epoch 4/5
209/209 [==============================] - 4s 19ms/step - loss: 1.6263 - acc: 0.2967 - val_loss: 2.0148 - val_acc: 0.1456
Epoch 5/5
209/209 [==============================] - 4s 21ms/step - loss: 1.4933 - acc: 0.3971 - val_loss: 2.2372 - val_acc: 0.1553
Training  1 th window.
Train on 209 samples, validate on 103 samples
Epoch 1/5
209/209 [==============================] - 5s 22ms/step - loss: 2.0762 - acc: 0.1627 - val_loss:

209/209 [==============================] - 4s 19ms/step - loss: 1.7885 - acc: 0.1914 - val_loss: 1.8088 - val_acc: 0.1650
Epoch 2/5
209/209 [==============================] - 4s 20ms/step - loss: 1.7888 - acc: 0.1914 - val_loss: 1.8104 - val_acc: 0.1650
Epoch 3/5
209/209 [==============================] - 4s 20ms/step - loss: 1.7855 - acc: 0.1914 - val_loss: 1.8073 - val_acc: 0.1650
Epoch 4/5
209/209 [==============================] - 4s 19ms/step - loss: 1.7888 - acc: 0.1914 - val_loss: 1.8075 - val_acc: 0.1650
Epoch 5/5
209/209 [==============================] - 4s 19ms/step - loss: 1.7845 - acc: 0.1914 - val_loss: 1.8083 - val_acc: 0.1553
Training  12 th window.
Train on 209 samples, validate on 103 samples
Epoch 1/5
209/209 [==============================] - 4s 19ms/step - loss: 1.7905 - acc: 0.1770 - val_loss: 1.8105 - val_acc: 0.1359
Epoch 2/5
209/209 [==============================] - 4s 19ms/step - loss: 1.7870 - acc: 0.1675 - val_loss: 1.8093 - val_acc: 0.1748
Epoch 3/5
209/20

In [ ]:
predict = model.predict(train_x_list[153])
predict = predict.tolist()
predict_labels = []
for i in range(len(predict)):
    predict_labels.append(predict[i].index(max(predict[i])))
print('Anger =',predict_labels.count(0))
print('Calmness =',predict_labels.count(1))
print('Disgust =',predict_labels.count(2))
print('Fear =',predict_labels.count(3))
print('Happiness =',predict_labels.count(4))
print('Sadness =',predict_labels.count(5))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()